### tf_yolov3 포멧(Mosaic Augmentation)
- tfrecord로 변환

In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "99"

import pandas as pd
import shutil
import cv2
from tqdm import tqdm
from glob import glob
from augmentation_pipeline import  Mosic_Augmentation
import tensorflow as tf


import random
import numpy as np

In [2]:
def _set_tf_features_by_mosaic(img4, labels4, label_names):
    
    succees, encoded_image = cv2.imencode('.jpg', np.array(img4))
    img_raw = encoded_image.tobytes()
    
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes_text = []
    height,width = (800,800)
    
    for label,x1,y1,x2,y2 in labels4:
        xmin.append(float(x1))
        ymin.append(float(y1))
        xmax.append(float(x2))
        ymax.append(float(y2))
        
        classes_text.append(label_names[int(label)].encode('utf8'))
        

    features={
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=['test'.encode('utf8')])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmin)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmax)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymin)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymax)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        }
    
    example = tf.train.Example(features=tf.train.Features(feature=features))
    
    return example

In [6]:
def create_mosaic_tfrecord(df, tf_path ,input_size, label_names, n_time=1):
    
    writer = tf.io.TFRecordWriter(tf_path)
    
    mosaic_border = [-input_size // 2, - input_size // 2]
    yc, xc = [int(random.uniform(-x, 2 * input_size + x)) for x in mosaic_border]  # mosaic center x, y
    
    mosic_aug = Mosic_Augmentation(input_size, mosaic_border, yc ,xc)
    
    images = df['img_path'].unique()
    n_images = len(images)    
    
    for _ in tqdm(range(n_time)):
        for img_path in images: 
            candidate_images = [cv2.imread(img_path)]
            
            labels = df[df['img_path']==img_path][['x1','y1','x2','y2','category_id']]
            labels = labels.to_numpy()
            candidate_labels = [np.hstack((labels[:,:-1]/800,labels[:,4:5]))]
            
            indices = [random.randint(0, n_images - 1) for _ in range(3)]  # 3 additional image indices
            for idx in indices:
                candidate_images.append(cv2.imread(images[idx]))
                
                labels = df[df['img_path']==images[idx]][['x1','y1','x2','y2','category_id']]
                labels = labels.to_numpy()
                candidate_labels = [np.hstack((labels[:,:-1]/800,labels[:,4:5]))]
            
            
            mosaic_i, mosaic_box = mosic_aug(candidate_images,candidate_labels)
        
            tfrecord_example = _set_tf_features_by_mosaic(mosaic_i, mosaic_box, label_names)
            writer.write(tfrecord_example.SerializeToString())

In [7]:
df = pd.read_csv('../../../DATA/googlemap/csv/label.csv')
label_names = {0 : '13',
               1 : '14',
               2 : '15',
               3 : '16',
               4 : '17',}

create_mosaic_tfrecord(df, './mosaic2.tfrecord' ,800, label_names, n_time=1)

100%|██████████| 1/1 [00:42<00:00, 42.78s/it]


In [8]:
!mv ./mosaic2.tfrecord ../../../DATA/googlemap/tfrecord/